<a href="https://colab.research.google.com/github/acse-mk1923/2dl/blob/main/genetic_cnnLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install deap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 3.5 MB/s eta 0:00:00


In [3]:
pip install tensorflow

In [44]:
import numpy as np
import random

from deap import base, creator, tools, algorithms
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Conv2D, MaxPooling2D, Flatten, Concatenate

# Set a random seed for reproducibility
np.random.seed(42)
random.seed(42)

# Custom CNN-LSTM model architecture
def create_cnn_lstm_model(input_shape_images, input_shape_data):
    # Image input branch
    image_input = Input(shape=input_shape_images)
    x = Conv2D(32, (3, 3), activation='relu')(image_input)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = Flatten()(x)
    image_branch = Model(image_input, x)

    # Data input branch
    data_input = Input(shape=input_shape_data)
    y = LSTM(64, activation='relu')(data_input)
    data_branch = Model(data_input, y)

    # Combine image and data branches
    combined = Concatenate()([image_branch.output, data_branch.output])
    z = Dense(64, activation='relu')(combined)
    z = Dense(1, activation='linear')(z)

    # Create the final model
    model = Model(inputs=[image_branch.input, data_branch.input], outputs=z)

    return model

# Constraint functions
def storage_capacity_constraint(solution):
    # Example constraint: total inventory volume should not exceed storage capacity
    total_volume = np.sum(solution)
    storage_capacity = 1000  # Assume a storage capacity of 1000 units
    return total_volume <= storage_capacity

def inventory_turnover_constraint(solution):
    # Example constraint: inventory turnover should be within a certain range
    inventory_turnover = 12  # Assume a desired inventory turnover of 12
    min_turnover = 10
    max_turnover = 14
    return min_turnover <= inventory_turnover <= max_turnover

def workstation_space_constraint(solution):
    # Example constraint: inventory at each workstation should not exceed available space
    workstation_space = 50  # Assume each workstation has a space of 50 units
    solution_values = []
    for individual in solution:
        if isinstance(individual, float):
            solution_values.append(individual)
        else:
            solution_values.extend(individual[:-1])
    return np.all(np.array(solution_values) <= workstation_space)

def material_flow_constraint(solution):
    # Example constraint: material flow should be balanced across workstations
    min_flow = 0.8
    max_flow = 1.2
    material_flow_ratio = np.max(solution) / np.min(solution)
    return min_flow <= material_flow_ratio <= max_flow

def finished_goods_storage_constraint(solution):
    # Example constraint: finished goods inventory should not exceed a certain percentage of total inventory
    finished_goods_percentage = 0.3  # Assume finished goods should not exceed 30% of total inventory
    total_inventory = np.sum(solution)
    finished_goods_inventory = solution[-1]
    return finished_goods_inventory <= finished_goods_percentage * total_inventory

def order_fulfillment_constraint(solution):
    # Example constraint: order fulfillment rate should be above a certain threshold
    order_fulfillment_threshold = 0.95  # Assume orders should be fulfilled at least 95% of the time
    order_fulfillment = 0.98  # Assume the current order fulfillment rate is 98%
    return order_fulfillment >= order_fulfillment_threshold

# Objective functions
def inventory_holding_cost(solution):
    # Example calculation: total holding cost based on unit cost and inventory levels
    unit_holding_cost = 0.1  # Assume a unit holding cost of 0.1 per unit per time period
    total_holding_cost = np.sum(solution) * unit_holding_cost
    return total_holding_cost

def stockout_risk(solution):
    # Example calculation: stockout risk based on safety stock levels and demand variability
    safety_stock_level = 100  # Assume a safety stock level of 100 units
    demand_variability = 0.2  # Assume a demand variability of 20%
    stockout_risk = np.exp(-solution[-1] / safety_stock_level) * demand_variability
    return stockout_risk

def inventory_accuracy(solution):
    # Example calculation: inventory accuracy based on actual and recorded inventory levels
    actual_inventory = np.sum(solution)
    recorded_inventory = actual_inventory * 0.95  # Assume 95% accuracy in inventory records
    inventory_accuracy = recorded_inventory / actual_inventory
    return inventory_accuracy

def workstation_inventory_efficiency(solution):
    # Example calculation: workstation inventory efficiency based on inventory levels and throughput
    target_inventory = 200  # Assume a target inventory level of 200 units per workstation
    actual_inventory = solution[:-1]
    inventory_efficiency = np.sum(actual_inventory) / (target_inventory * len(actual_inventory))
    return inventory_efficiency

def production_line_balance(solution):
    # Example calculation: production line balance based on workstation inventory levels
    target_balance = 0.9  # Assume a target balance of 90% across workstations
    actual_balance = np.min(solution[:-1]) / np.max(solution[:-1])
    balance_ratio = actual_balance / target_balance
    return balance_ratio

def order_fulfillment_rate(solution):
    # Example calculation: order fulfillment rate based on finished goods inventory and demand
    finished_goods_inventory = solution[-1]
    average_demand = 100  # Assume an average demand of 100 units per time period
    order_fulfillment_rate = finished_goods_inventory / average_demand
    return order_fulfillment_rate


# Corrected Fitness Functions
def fitness_function_storage_rooms(solution):
    if not storage_capacity_constraint(solution) or not inventory_turnover_constraint(solution):
        return np.inf, np.inf, -np.inf
    holding_cost = inventory_holding_cost(solution)
    stockout_risk_value = stockout_risk(solution)
    accuracy = inventory_accuracy(solution)
    # Ensure three values are returned
    return holding_cost, stockout_risk_value, accuracy

def fitness_function_factory_workstations(solution):
    if not workstation_space_constraint(solution) or not material_flow_constraint(solution):
        return -np.inf, -np.inf, np.inf  # Corrected to return three values
    inventory_efficiency = workstation_inventory_efficiency(solution)
    balance_ratio = production_line_balance(solution)
    # Adding a dummy third value (e.g., 1) to match the expected number of objectives
    return -inventory_efficiency, -balance_ratio, 1  # Assume a dummy value for the third objective

def fitness_function_completed_goods(solution):
    if not finished_goods_storage_constraint(solution) or not order_fulfillment_constraint(solution):
        return np.inf, -np.inf, np.inf  # Corrected to return three values
    holding_cost = inventory_holding_cost(solution)
    fulfillment_rate = order_fulfillment_rate(solution)
    # Adding a dummy third value (e.g., 1) for consistency
    return holding_cost, -fulfillment_rate, 1  # Assume a dummy value for the third objective



# Genetic algorithm setup
creator.create("FitnessMulti", base.Fitness, weights=(-1.0, -1.0, 1.0))
creator.create("Individual", list, fitness=creator.FitnessMulti)

toolbox = base.Toolbox()
toolbox.register("attr_float", np.random.uniform, low=0, high=1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=5)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Ensure the genetic algorithm setup uses the corrected fitness functions
toolbox.register("evaluate_storage_rooms", fitness_function_storage_rooms)
toolbox.register("evaluate_factory_workstations", fitness_function_factory_workstations)
toolbox.register("evaluate_completed_goods", fitness_function_completed_goods)

toolbox.register("mate", tools.cxSimulatedBinaryBounded, low=0.0, up=1.0, eta=20.0)
toolbox.register("mutate", tools.mutPolynomialBounded, eta=20.0, low=0.0, up=1.0, indpb=0.1)
toolbox.register("select", tools.selNSGA2)

# Synthetic data generation
def generate_synthetic_data(num_samples):
    # Generate synthetic image data
    image_data = np.random.rand(num_samples, 64, 64, 3)

    # Generate synthetic numerical data
    numerical_data = np.random.rand(num_samples, 10)

    # Reshape numerical data to have a dummy timestep dimension
    numerical_data = numerical_data.reshape(num_samples, 1, 10)

    # Generate synthetic labels
    labels = np.random.rand(num_samples)

    return image_data, numerical_data, labels

# Generate synthetic training data
num_training_samples = 1000
train_images, train_data, train_labels = generate_synthetic_data(num_training_samples)

# Train the CNN-LSTM model
train_images, val_images, train_data, val_data, train_labels, val_labels = train_test_split(train_images, train_data, train_labels, test_size=0.2)

cnn_lstm_model = create_cnn_lstm_model(input_shape_images=train_images.shape[1:], input_shape_data=train_data.shape[1:])
cnn_lstm_model.compile(optimizer='adam', loss='mse')
cnn_lstm_model.fit([train_images, train_data], train_labels, validation_data=([val_images, val_data], val_labels), epochs=10, batch_size=32)

# Genetic algorithm execution
def run_genetic_algorithm(toolbox, population_size, num_generations, crossover_prob, mutation_prob):
    population = toolbox.population(n=population_size)

    # Evaluate the fitness of the initial population
    fits = toolbox.map(toolbox.evaluate, population)
    for fit, ind in zip(fits, population):
        ind.fitness.values = fit  # Assign the tuple of fitness values directly

    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean, axis=0)
    stats.register("std", np.std, axis=0)
    stats.register("min", np.min, axis=0)
    stats.register("max", np.max, axis=0)

    logbook = tools.Logbook()
    logbook.header = ["gen"] + stats.fields

    for gen in range(num_generations):
        offspring = algorithms.varAnd(population, toolbox, cxpb=crossover_prob, mutpb=mutation_prob)
        fits = toolbox.map(toolbox.evaluate, offspring)
        for fit, ind in zip(fits, offspring):
            ind.fitness.values = fit  # Assign the tuple of fitness values directly
        population = toolbox.select(offspring + population, k=population_size)
        record = stats.compile(population)
        logbook.record(gen=gen, **record)

    return population, logbook
# Run the genetic algorithm for each task
population_size = 100
num_generations = 50
crossover_prob = 0.9
mutation_prob = 0.1

# Inventory management in storage rooms
toolbox.register("evaluate", toolbox.evaluate_storage_rooms)
storage_rooms_population, storage_rooms_logbook = run_genetic_algorithm(toolbox, population_size, num_generations, crossover_prob, mutation_prob)

# Inventory management at factory workstations
toolbox.register("evaluate", toolbox.evaluate_factory_workstations)
factory_workstations_population, factory_workstations_logbook = run_genetic_algorithm(toolbox, population_size, num_generations, crossover_prob, mutation_prob)

# Management of completed goods inventory
toolbox.register("evaluate", toolbox.evaluate_completed_goods)
completed_goods_population, completed_goods_logbook = run_genetic_algorithm(toolbox, population_size, num_generations, crossover_prob, mutation_prob)

# Print the best solutions for each task
print("Best solution for inventory management in storage rooms:")
print(storage_rooms_population[np.argmin([ind.fitness.values[0] for ind in storage_rooms_population])])

print("Best solution for inventory management at factory workstations:")
print(factory_workstations_population[np.argmax([ind.fitness.values[0] for ind in factory_workstations_population])])

print("Best solution for management of completed goods inventory:")
print(completed_goods_population[np.argmin([ind.fitness.values[0] for ind in completed_goods_population])])



Epoch 1/10
25/25 [==============================] - 3s 20ms/step - loss: 0.3805 - val_loss: 0.1157
Epoch 2/10
25/25 [==============================] - 0s 11ms/step - loss: 0.0900 - val_loss: 0.0874
Epoch 3/10
25/25 [==============================] - 0s 8ms/step - loss: 0.0850 - val_loss: 0.0881
Epoch 4/10
25/25 [==============================] - 0s 9ms/step - loss: 0.0852 - val_loss: 0.0874
Epoch 5/10
25/25 [==============================] - 0s 8ms/step - loss: 0.0863 - val_loss: 0.0876
Epoch 6/10
25/25 [==============================] - 0s 9ms/step - loss: 0.0853 - val_loss: 0.0946
Epoch 7/10
25/25 [==============================] - 0s 9ms/step - loss: 0.0850 - val_loss: 0.0916
Epoch 8/10
25/25 [==============================] - 0s 7ms/step - loss: 0.0860 - val_loss: 0.0914
Epoch 9/10
25/25 [==============================] - 0s 7ms/step - loss: 0.0851 - val_loss: 0.0872
Epoch 10/10
25/25 [==============================] - 0s 7ms/step - loss: 0.0892 - val_loss: 0.0890
Best solution for

In [40]:
# Set a random seed for reproducibility
np.random.seed(42)
random.seed(42)

# Synthetic data generation
def generate_synthetic_data(num_samples):
    # Generate synthetic image data
    image_data = np.random.rand(num_samples, 64, 64, 3)

    # Generate synthetic numerical data
    numerical_data = np.random.rand(num_samples, 10)

    # Reshape numerical data to have a dummy timestep dimension
    numerical_data = numerical_data.reshape(num_samples, 1, 10)

    # Generate synthetic labels
    labels = np.random.rand(num_samples)

    return image_data, numerical_data, labels

# Simulation Environment
class InventorySimulation:
    def __init__(self, image_data, numerical_data, labels, solution):
        self.image_data = image_data
        self.numerical_data = numerical_data
        self.labels = labels
        self.solution = solution

    def simulate(self):
        # Split the solution into components
        storage_rooms_inventory = self.solution[:4]
        workstations_inventory = self.solution[4:-1]
        finished_goods_inventory = self.solution[-1]

        # Corrected calculation of total_inventory_cost
        total_inventory_cost = np.sum(storage_rooms_inventory + workstations_inventory + [finished_goods_inventory]) * 0.1
        stockout_risk = np.exp(-finished_goods_inventory / 100) * 0.2  # Assume safety stock level of 100 and demand variability of 0.2
        inventory_accuracy = np.sum(storage_rooms_inventory + workstations_inventory) * 0.95 / np.sum(storage_rooms_inventory + workstations_inventory)  # Assume 95% accuracy

        # Check if workstations_inventory is empty
        if len(workstations_inventory) == 0:
            # Handle the empty case appropriately
            inventory_efficiency = 0  # Example default value
            production_balance = 0  # Example default value
        else:
            inventory_efficiency = np.sum(workstations_inventory) / (200 * len(workstations_inventory))  # Assume target inventory of 200 per workstation
            production_balance = np.min(workstations_inventory) / np.max(workstations_inventory)  # Production line balance

        order_fulfillment_rate = finished_goods_inventory / 100  # Assume average demand of 100 units

        return {
            'total_inventory_cost': total_inventory_cost,
            'stockout_risk': stockout_risk,
            'inventory_accuracy': inventory_accuracy,
            'inventory_efficiency': inventory_efficiency,
            'production_balance': production_balance,
            'order_fulfillment_rate': order_fulfillment_rate
        }

# Generate synthetic training data
num_training_samples = 1000
train_images, train_data, train_labels = generate_synthetic_data(num_training_samples)

# Train the CNN-LSTM model
train_images, val_images, train_data, val_data, train_labels, val_labels = train_test_split(train_images, train_data, train_labels, test_size=0.2)

cnn_lstm_model = create_cnn_lstm_model(input_shape_images=train_images.shape[1:], input_shape_data=train_data.shape[1:])
cnn_lstm_model.compile(optimizer='adam', loss='mse')
cnn_lstm_model.fit([train_images, train_data], train_labels, validation_data=([val_images, val_data], val_labels), epochs=10, batch_size=32)

# Genetic algorithm execution
def run_genetic_algorithm(toolbox, population_size, num_generations, crossover_prob, mutation_prob):
    population = toolbox.population(n=population_size)

    # Evaluate the fitness of the initial population
    fits = toolbox.map(toolbox.evaluate, population)
    for fit, ind in zip(fits, population):
        ind.fitness.values = fit  # Assign the tuple of fitness values directly

    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean, axis=0)
    stats.register("std", np.std, axis=0)
    stats.register("min", np.min, axis=0)
    stats.register("max", np.max, axis=0)

    logbook = tools.Logbook()
    logbook.header = ["gen"] + stats.fields

    for gen in range(num_generations):
        offspring = algorithms.varAnd(population, toolbox, cxpb=crossover_prob, mutpb=mutation_prob)
        fits = toolbox.map(toolbox.evaluate, offspring)
        for fit, ind in zip(fits, offspring):
            ind.fitness.values = fit  # Assign the tuple of fitness values directly
        population = toolbox.select(offspring + population, k=population_size)
        record = stats.compile(population)
        logbook.record(gen=gen, **record)

    return population, logbook

# Run the genetic algorithm for each task
population_size = 100
num_generations = 50
crossover_prob = 0.9
mutation_prob = 0.1

# Inventory management in storage rooms
toolbox.register("evaluate", toolbox.evaluate_storage_rooms)
storage_rooms_population, storage_rooms_logbook = run_genetic_algorithm(toolbox, population_size, num_generations, crossover_prob, mutation_prob)

# Evaluate the best solution for inventory management in storage rooms
best_solution_storage_rooms = storage_rooms_population[np.argmin([ind.fitness.values[0] for ind in storage_rooms_population])]
print("Best solution for inventory management in storage rooms:")
print(best_solution_storage_rooms)

# Run the simulation for the best solution
simulation = InventorySimulation(train_images, train_data, train_labels, best_solution_storage_rooms)
simulation_results = simulation.simulate()

# Print the simulation results
print("\nSimulation Results for Best Solution (Storage Rooms):")
for metric, value in simulation_results.items():
    print(f"{metric}: {value}")

# Inventory management at factory workstations
toolbox.register("evaluate", toolbox.evaluate_factory_workstations)
factory_workstations_population, factory_workstations_logbook = run_genetic_algorithm(toolbox, population_size, num_generations, crossover_prob, mutation_prob)

# Evaluate the best solution for inventory management at factory workstations
best_solution_factory_workstations = factory_workstations_population[np.argmax([ind.fitness.values[0] for ind in factory_workstations_population])]
print("\nBest solution for inventory management at factory workstations:")
print(best_solution_factory_workstations)

# Run the simulation for the best solution
simulation = InventorySimulation(train_images, train_data, train_labels, best_solution_factory_workstations)
simulation_results = simulation.simulate()

# Print the simulation results
print("\nSimulation Results for Best Solution (Factory Workstations):")
for metric, value in simulation_results.items():
    print(f"{metric}: {value}")

# Management of completed goods inventory
toolbox.register("evaluate", toolbox.evaluate_completed_goods)
completed_goods_population, completed_goods_logbook = run_genetic_algorithm(toolbox, population_size, num_generations, crossover_prob, mutation_prob)

# Evaluate the best solution for management of completed goods inventory
best_solution_completed_goods = completed_goods_population[np.argmin([ind.fitness.values[0] for ind in completed_goods_population])]
print("\nBest solution for management of completed goods inventory:")
print(best_solution_completed_goods)

# Run the simulation for the best solution
simulation = InventorySimulation(train_images, train_data, train_labels, best_solution_completed_goods)
simulation_results = simulation.simulate()

# Print the simulation results
print("\nSimulation Results for Best Solution (Completed Goods Inventory):")
for metric, value in simulation_results.items():
    print(f"{metric}: {value}")



Epoch 1/10
25/25 [==============================] - 3s 19ms/step - loss: 0.3411 - val_loss: 0.0898
Epoch 2/10
25/25 [==============================] - 0s 8ms/step - loss: 0.0866 - val_loss: 0.0873
Epoch 3/10
25/25 [==============================] - 0s 8ms/step - loss: 0.0863 - val_loss: 0.0875
Epoch 4/10
25/25 [==============================] - 0s 7ms/step - loss: 0.0885 - val_loss: 0.0915
Epoch 5/10
25/25 [==============================] - 0s 7ms/step - loss: 0.0886 - val_loss: 0.0914
Epoch 6/10
25/25 [==============================] - 0s 7ms/step - loss: 0.0887 - val_loss: 0.0906
Epoch 7/10
25/25 [==============================] - 0s 7ms/step - loss: 0.0867 - val_loss: 0.0884
Epoch 8/10
25/25 [==============================] - 0s 7ms/step - loss: 0.0894 - val_loss: 0.0878
Epoch 9/10
25/25 [==============================] - 0s 7ms/step - loss: 0.0853 - val_loss: 0.0908
Epoch 10/10
25/25 [==============================] - 0s 7ms/step - loss: 0.0844 - val_loss: 0.0875
Best solution for 